In [36]:
import cv2, glob, pickle, csv
import numpy as np
import matplotlib.pyplot as plt

In [37]:
image_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/ISIC-2017/ISIC-2017/"
image_extension = ".jpg"

num_image_files = 0
pixel_depth = 255.0

resized_folder_path = "/home/openroot/Tanmoy/Working Stuffs/myStuffs/havss-tf/ISIC-2017/resized/"
resized_image_size = (100, 70)

In [38]:
def resizeImage():
    image_files = glob.glob(image_folder_path + "*" + image_extension)
    for image in image_files:
        try:
            image_name = image.split("/")[-1]
            image = cv2.imread(image, cv2.IMREAD_UNCHANGED)
            image = cv2.resize(image, resized_image_size)
            cv2.imwrite(resized_folder_path +  image_name, image)
        except Exception as e:
            print("Unable To Reize {0}".format(image_name))

In [39]:
def imagePickle(images):
    try:
        pickle_file = resized_folder_path + "imageDataset.pickle"
        with open(pickle_file, "wb") as f:
            pickle.dump(images, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print("Error While Creating Pickle File {0}".format(e))

In [40]:
def loadPickle():
    try:
        pickle_file = resized_folder_path + "imageDataset.pickle"
        with open(pickle_file, "rb") as f:
            dataset = pickle.load(f)
    except Exception as e:
        print("Error While Loading Pickle File {0}".format(e))
    return dataset

In [41]:
def readCSV():
    label_dataset = []
    label_names = []
    label_file = resized_folder_path + "ISIC-2017-label.csv"
    try:
        with open(label_file, "rb") as f:
            reader = csv.DictReader(f)
            for row in reader:
                label_names = np.append(label_names, row["image_id"])
                if np.array_equal([float(row["melanoma"]), float(row["seborrheic_keratosis"])], [0., 0.]):
                    label_dataset = np.append(label_dataset, [1., 0., 0.])
                elif np.array_equal([float(row["melanoma"]), float(row["seborrheic_keratosis"])], [0., 1.]):
                    label_dataset = np.append(label_dataset, [0., 1., 0.])
                else: 
                    label_dataset = np.append(label_dataset, [0., 0., 1.])
#                 label_dataset = np.append(label_dataset, [float(row["melanoma"]), float(row["seborrheic_keratosis"])])
            label_dataset = np.reshape(label_dataset, (-1,3))
    except Exception as e:
        print("Error While Reading CSV Label File: {0}".format(e))
    return label_dataset, label_names

In [42]:
def createDataset(image_dataset, csv_data, percent_test = 5):
    total_images = len(image_dataset)
    print("Total No. Of Images {0}".format(total_images))
    no_test = 300
    print("Division Of Dataset {0}".format(no_test))
    test_images = image_dataset[0:no_test, :, :, :]
    test_labels = csv_data[0:no_test, :]
    train_images = image_dataset[no_test:, :, :, :]
    train_labels = csv_data[no_test:, :]
    return train_images, train_labels, test_images, test_labels

In [51]:
def main():
    print("Resizing Images")
#     resizeImage()
    
    print("Reading CSV Label File")
    csv_data, csv_data_names = readCSV()
    print("Lenght Of CSV File {0}".format(len(csv_data)))
#     print csv_data_names
    
    print("Creating Image Array")
#     image_files = glob.glob(resized_folder_path + "*" + image_extension)
    image_x = np.array([(cv2.imread(resized_folder_path + image + image_extension).astype(float) - pixel_depth / 2) / pixel_depth for image in csv_data_names])
    print image_x.shape
#     print image_files
    
    print("Saving Image Array In Pickle Form - imageDataset.pickle")
    imagePickle(image_x)
    
#     print image_x[0][2][0]
#     image_x[0][2][0][0] = 0
#     print image_x[0][2][0]
    
#     Check Equality
#     dataset = loadPickle()
#     print np.array_equal(image_x, dataset)

    print("Creating Training And Testing Dataset")
    train_images, train_labels, test_images, test_labels = createDataset(image_x, csv_data)
    
    print("Length Of Training Image Data {0} Shape {1}".format(len(train_images), train_images.shape))
    print("Length Of Training Label Data {0} Shape {1}".format(len(train_labels), train_labels.shape))
    print("Length Of Test Image Data {0} Shape {1}".format(len(test_images), test_images.shape))
    print("Length Of Test Label Data {0} Shape {1}".format(len(test_labels), test_labels.shape))
    
#     print("Saving All Dataset In Pickle Form - ISIC.pickle")
    pickle_file = resized_folder_path + "ISIC.pickle"
    try:
        with open(pickle_file, "wb") as f:
            save = {
                'train_dataset': train_images,
                'train_labels': train_labels,
                'test_dataset': test_images,
                'test_labels': test_labels,
            }
            pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print("Error While Saving ISIC Pickle {0}".format(e))
        
    print csv_data[0:10, :]
    print test_labels[0:10, :]

In [52]:
main()

Resizing Images
Reading CSV Label File
Lenght Of CSV File 1988
Creating Image Array
(1988, 70, 100, 3)
Saving Image Array In Pickle Form - imageDataset.pickle
Creating Training And Testing Dataset
Total No. Of Images 1988
Division Of Dataset 300
Length Of Training Image Data 1688 Shape (1688, 70, 100, 3)
Length Of Training Label Data 1688 Shape (1688, 3)
Length Of Test Image Data 300 Shape (300, 70, 100, 3)
Length Of Test Label Data 300 Shape (300, 3)
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]
[[ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]]
